#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
Purpose: Script to create using the netapp_ontap library.
         It will create an SVM.
         It will create an NFS server.
Author: Vish Hulikal
Usage: nas_nfs.py [-h] -c CLUSTER -vs VSERVER_NAME, -a AGGR_NAME,  -d DOMAIN, -s SERVER_IP,
                    -se NFS_SERVER, [-u API_USER] [-p API_PASS]
"""


#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.utils`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/utils.html) : This module contains some of the common utility functions used in the library
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.svm`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/svm.html) : Managing SVMs
  - [`netapp_ontap.resources.nfs_service`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/nfs_service.html) 
  

In [ ]:

import argparse
from getpass import getpass
import logging
from typing import Optional

from netapp_ontap import config, utils, HostConnection, NetAppRestError
from netapp_ontap.resources import Svm, NfsService, ExportPolicy, ExportRule


#### Function Definitions

These functions are defined:
  - `create_svm`
  - `create_nfs_server`


In [ ]:

def create_svm(vserver_name: str, aggr_name: str) -> None:
    """Create an SVM on the specified aggregate"""

    svm = Svm.from_dict({
    'name': vserver_name,
    'aggregates': [{'name': aggr_name}],
    'nfsv3': {'enabled': "true"},
    'nfsv4': {'enabled': "false"},
    'nfsv41': {'enabled': "false"}
    })

    try:
        svm.post()
        print("SVM %s created successfully" % svm.name)
    except NetAppRestError as err:
        print("Error: SVM was not created: %s" % err)
    return



In [ ]:
def create_nfs_server(vserver_name: str, domain_name: str, nfs_server: str, server_ip: str) -> None:
    """Creates a NFS server"""

    SVM = Svm.find(name=vserver_name)

    data = {
        'name': nfs_server,
        'scope': 'svm',
        'svm': {'name': vserver_name, 'uuid': SVM.uuid},
        'protocol': {'v4_id_domain': domain_name},
        'vstorage_enabled': 'true'
    }

    nfs_service = NfsService(**data)

    try:
        nfs_service.post()
        print("NFS Server %s created successfully" % nfs_server)
    except NetAppRestError as err:
        print("Error: NFS Server was not created: %s" % err)
    return



#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a new NFS VServer"
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="Cluster Name"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="VServer name to create NFS Share"
    )
    parser.add_argument(
        "-a", "--aggr_name", required=True, help="Aggregate name"
    )
    parser.add_argument(
        "-d", "--domain", required=True, help="DNS DOmain Name"
    )
    parser.add_argument(
        "-s", "--server_ip", required=True, help="DNS Server IP Address"
    )
    parser.add_argument(
        "-se", "--nfs_server", required=True, help="NFS Server"
    )

    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args



#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
    #    level=logging.DEBUG,
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )
    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )

    create_svm(args.vserver_name, args.aggr_name)
    create_nfs_server(args.vserver_name, args.domain, args.nfs_server, args.server_ip)


[Return to the Module 4 Notebook:](module4.ipynb)